# Autograd: automatic differentiation

pytorch에서 만들 수 있는 모든 neural network들의 핵심은 `autograd` 패키지이다.  
  
`autograd` 패키지는 모든 Tensor 연산에 대한 변화량을 자동으로 계산하는 방법을 제공해준다.

## Variable

`autograd.Variable`은 `autograd` 패키지의 핵심이다. Tensor를 감싸는 역할을 하고, Tensor가 할 수 있는 연산을 거의 다 지원한다. 연산을 다 하고 `backward` 메서드를 호출하면 변화량을 자동으로 계산해준다.

Variable로 감싼 Tensor를 가져오고 싶으면 `data` 멤버 변수를, 변화량을 가져오고 싶으면 `grad` 멤버 변수를 이용한다.

`Function`도 엄청 중요한 class이다. Variable과 Function은 서로 상호작용을 통해 acyclic graph를 만들게 된다. 사용자가 만든 Variable을 제외한 모든 Variable은 `grad_fn`이라는 멤버 변수를 갖는데, 자신을 만든 Function을 가리키는 변수이다.

미분을 통해 변화량을 얻고 싶다면 Variable의 `backward` 메서드를 이용하면 된다. 만약 Variable이 스칼라(하나의 값만을 지님)면 변화량을 얻고 싶은 수를 지정할 필요가 없지만, 많은 수의 값들을 지니고 있다면 `backward` 메서드의 parameter를 통해 지정해야 한다.

In [1]:
import torch
from torch.autograd import Variable

Variable을 만드려면

In [2]:
x = Variable(torch.ones(2, 2), requires_grad=True)
print(x)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



Variable에 대해 연산은 아래와 같다.

In [3]:
y = x + 2
print(y)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



`y`는 사용자가 만든게 아니라 연산을 통해 만들어졌기 때문에 `grad_fn`이라는 멤버 변수가 존재한다.

In [4]:
print(y.grad_fn)

y를 이용해 다른 연산을 해보자.

In [5]:
z = y * y * 3
out = z.mean()

print(z)
print(out)

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]

Variable containing:
 27
[torch.FloatTensor of size 1]



## Gradients

`backward` 메서드를 통해 Backpropagation을 해보자. 위에서 만든 `out`을 이용해 `out.backward()`를 해보자. 인자없이 실행하면 인자가 `torch.Tensor([1.0])`이라는 것과 같다.

In [6]:
out.backward()

In [7]:
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



out이 y일 때 out의 식은 아래와 같다.

$$
y = {1 \over 4}\sum_iz_i \\ z_i = 3(x_i + 2)^2
$$

x가 1일 때 z는 27이 된다. 따라서 x에 대한 y의 미분은

$$
{\partial y \over \partial x_i} = {3 \over 2}(x_i + 2)
$$

가 되서 x가 1일 때 4.5가 된다.

이외에도 재미난 많은 일들을 할 수 있다.

In [9]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2
    
print(y)

Variable containing:
 -484.2421
 -236.1657
-1569.2457
[torch.FloatTensor of size 3]



In [10]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
# backward 메서드의 인자는 반드시 크기가 y와 같아야 하는데, 
# 이유는 y안의 값과 element-wise로 매칭되는 인자 안의 값이 미분할 때 사용되기 때문이다.  
y.backward(gradients)

print(x.grad)

Variable containing:
  204.8000
 2048.0000
    0.2048
[torch.FloatTensor of size 3]

